## Imports

In [ ]:
#!pip install db-dtypes google google-cloud google-cloud-bigquery google-auth

In [1]:
import pandas as pd
import numpy as np
import math
from google.cloud import bigquery
from google.oauth2 import service_account
import json
import datetime
from datetime import datetime

## Data Load

In [2]:
with open('ck-case-study-c83094e812a0.json', 'r') as json_file:
    data = json.load(json_file)

In [3]:
def read_bigquery_data(query,data):
    # Create a BigQuery client instance
    credentials = service_account.Credentials.from_service_account_info(data)
    client = bigquery.Client(credentials=credentials)

    # Execute the query and retrieve the results as a pandas DataFrame
    query_job = client.query(query)
    dataframe = query_job.to_dataframe()

    return dataframe

In [4]:
query_orders='''SELECT * FROM `ck-case-study.ck_us_can_case_studies.order_data`'''

query_labor = '''SELECT * FROM `ck-case-study.ck_us_can_case_studies.labor_hours`'''

df_orders = read_bigquery_data(query_orders,data)
df_labor = read_bigquery_data(query_labor,data)

In [5]:
df_orders.to_csv('data_orders.csv', index=False)
df_labor.to_csv('data_labor.csv', index=False)

## Cleaning

In [2]:
data_orders = pd.read_csv('data_orders.csv')

data_orders = data_orders.rename(columns=lambda x: x.lower())
data_orders["date"] = pd.to_datetime(data_orders["date"],format="%Y-%m-%d")

In [3]:
# We only have data for August and a few days of September 2023
data_orders.date.describe()

count                         50000
mean     2023-08-19 23:00:24.768000
min             2023-08-01 00:00:00
25%             2023-08-11 00:00:00
50%             2023-08-20 00:00:00
75%             2023-08-29 00:00:00
max             2023-09-08 00:00:00
Name: date, dtype: object

In [4]:
data_orders.shape

(50000, 9)

In [5]:
# we remove duplicate values and Na values
data_orders.drop_duplicates(keep ='last', inplace = True)
data_orders.dropna(inplace = True)

In [6]:
data_orders.shape

(46546, 9)

In [7]:
# one restaurant can be in several facilities
data_orders.loc[data_orders['organization_name']=='witty piano']['facility'].value_counts()

facility
NA-US-NG-OTT-Tempe (Kitchens) (correct)    1402
NA-US-OD-5th (correct)                      234
Name: count, dtype: int64

In [8]:
# We remove all rows with a number of orders>0 but gmv=0 since it does not make sense 
data_orders = data_orders.loc[(data_orders['orders']>0)&(data_orders['gmv']>0)]
data_orders.shape

(45976, 9)

In [9]:
data_orders['week'] = data_orders['date'] - pd.to_timedelta(data_orders['date'].dt.dayofweek, unit='d')
data_orders['month'] = data_orders['date'].dt.to_period('M').dt.to_timestamp()

In [10]:
data_orders.to_csv('data_orders_clean.csv', index=False)

In [11]:
#We group the data by day since there are several rows duplicated by day that come from the same facility and restaurant
data_orders_grouped = data_orders.groupby(['date','facility','facility_id',
                                'organization_name','subregion'])[['gmv','orders','gmv_minus_discount']].sum().reset_index()

data_orders_grouped.shape

(3312, 8)

In [12]:
data_orders_grouped['week'] = data_orders_grouped['date'] - pd.to_timedelta(data_orders_grouped['date'].dt.dayofweek, unit='d')
data_orders_grouped['month'] = data_orders_grouped['date'].dt.to_period('M').dt.to_timestamp()

In [13]:
data_orders_grouped.to_csv('data_orders_grouped.csv', index=False)

In [14]:
data_orders_grouped.head()

,date,facility,facility_id,organization_name,subregion,gmv,orders,gmv_minus_discount,week,month
0,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,abundant river,Northwest,1807.00,44,1722.11,2023-07-31,2023-08-01
1,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,agile tiger,Northwest,33.99,1,33.99,2023-07-31,2023-08-01
2,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,alluring dolphin,Northwest,1110.95,38,1104.95,2023-07-31,2023-08-01
3,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,charming island,Northwest,30.00,1,30.00,2023-07-31,2023-08-01
4,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,colorful kangaroo,Northwest,12.40,1,12.40,2023-07-31,2023-08-01


In [15]:
data_labor = pd.read_csv('data_labor.csv')
data_labor.head()

,date,facility_id,facility_name,labor_hours_actual_including_cr_hours_allocation,daily_cr_labor_hours_allocation
0,2023-08-26,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,0.000000,0.0
1,2023-08-05,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,0.000000,0.0
2,2023-08-15,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,7.416667,0.0
3,2023-08-01,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,0.000000,0.0
4,2023-08-31,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,9.616667,0.0


In [16]:
data_labor.shape

(266, 5)

In [17]:
data_labor["date"] = pd.to_datetime(data_labor["date"],format="%Y-%m-%d")

In [18]:
#We do not have duplicates
data_labor.drop_duplicates(keep='last')
data_labor.shape

(266, 5)

In [19]:
# No Na values
data_labor.dropna(inplace=True)
data_labor.shape

(266, 5)

In [20]:
data_labor['week'] = data_labor['date'] - pd.to_timedelta(data_labor['date'].dt.dayofweek, unit='d')
data_labor['month'] = data_labor['date'].dt.to_period('M').dt.to_timestamp()

In [21]:
data_labor.rename(columns = {'facility_name':'facility',
'labor_hours_actual_including_cr_hours_allocation':'hours_staffed'}, inplace=True)

data_labor['labor_cost'] = data_labor['hours_staffed']*18

In [22]:
data_labor.to_csv('data_labor_clean.csv', index=False)

In [23]:
data_labor.head()

,date,facility_id,facility,hours_staffed,daily_cr_labor_hours_allocation,week,month,labor_cost
0,2023-08-26,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,0.000000,0.0,2023-08-21,2023-08-01,0.000000
1,2023-08-05,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,0.000000,0.0,2023-07-31,2023-08-01,0.000000
2,2023-08-15,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,7.416667,0.0,2023-08-14,2023-08-01,133.500006
3,2023-08-01,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,0.000000,0.0,2023-07-31,2023-08-01,0.000000
4,2023-08-31,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,NA-US-CO-DEN-Aurora,9.616667,0.0,2023-08-28,2023-08-01,173.100006


In [24]:
data_all = pd.merge(data_orders_grouped,data_labor, on=['date','facility_id'],how='left')
data_all.shape

(3312, 16)

In [26]:
# WE CALCULATE THE LABOR COST, ASSUMING WE PAY RUNNERS $18 PER HOUR
data_all['labor_cost'] = data_all['hours_staffed']*18

In [27]:
data_all.head()

,date,facility_x,facility_id,organization_name,subregion,gmv,orders,gmv_minus_discount,week_x,month_x,facility_y,hours_staffed,daily_cr_labor_hours_allocation,week_y,month_y,labor_cost
0,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,abundant river,Northwest,1807.00,44,1722.11,2023-07-31,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),29.483333,0.0,2023-07-31,2023-08-01,530.699994
1,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,agile tiger,Northwest,33.99,1,33.99,2023-07-31,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),29.483333,0.0,2023-07-31,2023-08-01,530.699994
2,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,alluring dolphin,Northwest,1110.95,38,1104.95,2023-07-31,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),29.483333,0.0,2023-07-31,2023-08-01,530.699994
3,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,charming island,Northwest,30.00,1,30.00,2023-07-31,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),29.483333,0.0,2023-07-31,2023-08-01,530.699994
4,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,colorful kangaroo,Northwest,12.40,1,12.40,2023-07-31,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),29.483333,0.0,2023-07-31,2023-08-01,530.699994


## Metrics

In [28]:
# TOP 5 MOST SUCCESSFUL RESTAURANTS
data_orders_grouped.groupby(['organization_name'])[['gmv','orders']].sum().sort_values(by =['gmv'],
ascending=False).reset_index().head(5)

,organization_name,gmv,orders
0,boring apple,388170.78,11094
1,witty piano,192575.13,6677
2,abundant river,186832.91,6317
3,delicious tiger,185326.25,6864
4,alluring dolphin,126175.95,4908


In [30]:
#AVERAGE NUMBER OF ORDERS AND GMV PER DAY PER RESTAURANT AND FACILITY
#HERE IN THIS WAY WE ARE PRETTY MUCH CALCULATING THE AVG #ORDERS BY DAY, EVERYTHING BY DAY
# WE COULD CHANDE THE TIME PERIOD TO WEEK OR MONTH

data_orders_grouped.groupby(['organization_name'])[['gmv','gmv_minus_discount','orders']].mean().reset_index()

,organization_name,gmv,gmv_minus_discount,orders
0,abundant apple,151.350588,131.117059,4.411765
1,abundant bicycle,816.170000,687.591316,26.842105
2,abundant cloud,98.921875,87.824062,3.343750
3,abundant dolphin,25.164783,24.926522,1.304348
4,abundant elephant,409.007353,382.743529,15.852941
...,...,...,...,...
125,vibrant zebra,192.277333,156.298000,6.933333
126,witty house,346.420263,315.278947,13.052632
127,witty laptop,396.940789,354.279474,12.315789
128,witty ocean,19.127143,19.057143,1.428571


In [31]:
#AVERAGE AMOUNT OF ORDERS AND GMV PER DAY 
data_orders.loc[data_orders['organization_name']=='boring apple'].groupby(['facility','organization_name'])[['gmv','gmv_minus_discount','orders']].mean().reset_index()

,facility,organization_name,gmv,gmv_minus_discount,orders
0,NA-US-CA(N)-NH-Ruff (Kitchens),boring apple,37.121053,36.414211,1.157895
1,NA-US-IL-DUN-Avondale,boring apple,209.817319,138.021964,5.953927
2,NA-US-OD-5th (correct),boring apple,63.945977,49.281557,1.859475


In [32]:
# AVERAGE NUMBER OF ORDERS AND GMV ONLY PER FACILITY 

data_orders.groupby(['facility'])[['gmv','gmv_minus_discount','orders']].mean().sort_values(by=['orders'],ascending = False).reset_index()

,facility,gmv,gmv_minus_discount,orders
0,NA-US-IL-DUN-Avondale,69.950111,57.231615,2.288638
1,NA-US-TX-KOL-18th,66.866208,61.440913,2.175258
2,NA-US-CA(N)-NH-Ruff (Kitchens),66.052421,64.206937,2.048513
3,NA-US-NG-OTT-Tempe (Kitchens) (correct),59.260392,49.312910,1.997836
4,NA-US-OD-5th (correct),51.363129,45.597707,1.680794
5,NA-US-TUCS-SA-Lombrano (Correct),40.948912,39.821425,1.456354
6,NA-US-CO-DEN-Aurora,40.082818,35.284280,1.244145


In [34]:
# DISTRIBUTION OF HOURS STAFFED IN THE FACILITIES DURING THIS PERIOD OF TIME
data_labor.hours_staffed.describe()

count    266.000000
mean      10.886967
std        9.077066
min        0.000000
25%        0.000000
50%       10.675000
75%       17.258333
max       34.466668
Name: hours_staffed, dtype: float64

In [37]:
# AVG #ORDERS PER HOUR FOR EACH FACILITY
# ASUMING WE HAVE 11 OPERATION HOURS ON A REGULAR DAY (BASED ON AVG)
# AND INFORMATION OF 39 DAYS

orders_facility = data_orders.groupby(['facility'])[['orders']].sum().reset_index()

orders_facility['avg_orders_perhour'] = orders_facility['orders']/(11*39)

orders_facility.sort_values(by=['avg_orders_perhour'],ascending=False)

,facility,orders,avg_orders_perhour
2,NA-US-IL-DUN-Avondale,26388,61.510490
3,NA-US-NG-OTT-Tempe (Kitchens) (correct),20310,47.342657
4,NA-US-OD-5th (correct),18877,44.002331
0,NA-US-CA(N)-NH-Ruff (Kitchens),17355,40.454545
6,NA-US-TX-KOL-18th,2954,6.885781
5,NA-US-TUCS-SA-Lombrano (Correct),2636,6.144522
1,NA-US-CO-DEN-Aurora,1753,4.086247


In [40]:
# WE CALCULATE THE LABOR COST, ASSUMING WE PAY RUNNERS $18 PER HOUR
#data_labor['labor_cost'] = data_labor['labor_hours_actual_including_cr_hours_allocation']*18

# AVG LABOR HOUR AND LABOR COST PER FACILITY (AT DAY LEVEL)

data_labor.groupby(['facility'])[['hours_staffed','labor_cost']].mean().sort_values(by=['labor_cost'],ascending=False).reset_index()

,facility,hours_staffed,labor_cost
0,NA-US-CA(N)-NH-Ruff (Kitchens),19.890790,358.034213
1,NA-US-NG-OTT-Tempe (Kitchens) (correct),18.712281,336.821055
2,NA-US-OD-5th (correct),14.622368,263.202630
3,NA-US-IL-DUN-Avondale,12.175439,219.157893
4,NA-US-TUCS-SA-Lombrano (Correct),5.030702,90.552631
5,NA-US-CO-DEN-Aurora,3.015351,54.276316
6,NA-US-TX-KOL-18th,2.761842,49.713157


In [43]:
# WE HAVE FACILITIES IN CERTAIN DAYS , IN WHICH WE DIDN'T STAFFED ANY HOURS, BUT CHECKING THE 
# NUMBER OF ORDERS PER RESTAURANT, SEEMS TO BE OK

data_all.loc[(data_all['date']=='2023-08-26')&(data_all['facility_x']=='NA-US-CO-DEN-Aurora')]

,date,facility_x,facility_id,organization_name,subregion,gmv,orders,gmv_minus_discount,week_x,month_x,facility_y,hours_staffed,daily_cr_labor_hours_allocation,week_y,month_y,labor_cost
2191,2023-08-26,NA-US-CO-DEN-Aurora,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,abundant bicycle,Northwest,927.00,33,822.10,2023-08-21,2023-08-01,NA-US-CO-DEN-Aurora,0.0,0.0,2023-08-21,2023-08-01,0.0
2192,2023-08-26,NA-US-CO-DEN-Aurora,2019e0f4-7fd8-47fd-a858-9e5c4b409a39,optimistic flower,Northwest,927.03,29,857.11,2023-08-21,2023-08-01,NA-US-CO-DEN-Aurora,0.0,0.0,2023-08-21,2023-08-01,0.0


In [44]:
# HERE WE CAN SUPPORT OUR PREVIOUS POINT EVEN BETTER
data_all.loc[data_all['hours_staffed']==0]['orders'].describe()

count    490.000000
mean      22.502041
std       34.694474
min        1.000000
25%        2.000000
50%        9.000000
75%       27.000000
max      257.000000
Name: orders, dtype: float64

In [74]:
#At which facilities are we earning vs losing money considering labor costs and processing revenue? 
#And which days are not correctly staffed (understafffed or overstaffed)?

data_facility = data_orders.loc[data_orders['date']<'2023-09-08'].groupby(['date','facility','facility_id'],
                        as_index=False)[['orders','gmv','gmv_minus_discount']].sum()

data_facility = pd.merge(data_facility, data_labor.drop(columns=['facility']), on=['date','facility_id'],how='left')

data_facility.drop(columns=['daily_cr_labor_hours_allocation'], inplace=True)

#data_facility['hours_staffed'] = data_facility['hours_staffed'].apply(lambda x : math.ceil(x))

# THIS IS THE NUMBER OF ORDERS PROCESSED PER HOUR STAFFED AT EACH FACILITY
data_facility['orders_processed_per_hour_staffed'] = np.where(data_facility['hours_staffed']==0,0,data_facility['orders']/data_facility['hours_staffed'])
data_facility['orders_processed_per_hour_staffed'] = data_facility['orders_processed_per_hour_staffed'].apply(lambda x : math.ceil(x))

# THIS IS THE AMOUNT OF HOURS WE SHOULD HAVE STAFFED CONSIDERING THE NUMBER OF ORDERS
data_facility['expected_hours'] = data_facility['orders']/35
#data_facility['expected_hours'] = data_facility['expected_hours'].apply(lambda x : math.ceil(x))

data_facility['overstaffing_flag'] = np.where(data_facility.hours_staffed>data_facility.expected_hours,1,0)

#data_facility['understaffing_flag'] = np.where((data_facility['hours_staffed']==0)&(data_facility['orders']>=35),1,0)

data_facility['processing_revenue'] = data_facility['gmv']*0.04

data_facility['net_revenue'] = data_facility['processing_revenue'] - data_facility['labor_cost']
data_facility['expected_labor_cost'] = data_facility['expected_hours']*18
data_facility['expected_net_revenue'] =data_facility['processing_revenue'] - data_facility['expected_labor_cost'] 

data_facility['loss_flag'] = np.where(data_facility['net_revenue']<0,1,0)

data_facility.shape

(265, 18)

In [75]:
data_facility.head()

,date,facility,facility_id,orders,gmv,gmv_minus_discount,hours_staffed,week,month,labor_cost,orders_processed_per_hour_staffed,expected_hours,overstaffing_flag,processing_revenue,net_revenue,expected_labor_cost,expected_net_revenue,loss_flag
0,2023-08-01,NA-US-CA(N)-NH-Ruff (Kitchens),905cbe43-327f-4d3e-a897-48ee81a64ade,154,5478.43,5322.32,29.483333,2023-07-31,2023-08-01,530.699994,6,4.400000,1,219.1372,-311.562794,79.200000,139.937200,1
1,2023-08-01,NA-US-IL-DUN-Avondale,54341152-da81-4b58-8ba3-8604bc1ff2dc,108,3297.44,2555.35,13.200000,2023-07-31,2023-08-01,237.600000,9,3.085714,1,131.8976,-105.702400,55.542857,76.354743,1
2,2023-08-01,NA-US-NG-OTT-Tempe (Kitchens) (correct),a9eb0fc1-7699-43b4-bd4c-6bb5b8195a87,69,1949.61,1850.56,13.166667,2023-07-31,2023-08-01,237.000006,6,1.971429,1,77.9844,-159.015606,35.485714,42.498686,1
3,2023-08-01,NA-US-OD-5th (correct),54d4375f-4c79-4444-ad83-f54362c08198,134,4088.62,3712.86,12.583333,2023-07-31,2023-08-01,226.499994,11,3.828571,1,163.5448,-62.955194,68.914286,94.630514,1
4,2023-08-01,NA-US-TUCS-SA-Lombrano (Correct),ee395d7f-589b-4208-a745-90a8a0aa3bfd,12,286.02,266.02,8.066667,2023-07-31,2023-08-01,145.200006,2,0.342857,1,11.4408,-133.759206,6.171429,5.269371,1


In [76]:
data_facility.to_csv('data_facility.csv', index=False)

In [69]:
# HAVING THE ASSUMPTION THAT A RUNNER CAN PROCESS 35 ORDERS PER HOUR AND WE TAKE A LOOK PER DAY
# WE ARE OVERSTAFFING IN A ~51% OF THE DAYS

data_facility.overstaffing_flag.value_counts(1)*100

overstaffing_flag
1    51.320755
0    48.679245
Name: proportion, dtype: float64

In [70]:
# WE ARE LOOSING MONEY AT ~21% OF THE DAYS
data_facility.loss_flag.value_counts(1)*100

loss_flag
0    79.245283
1    20.754717
Name: proportion, dtype: float64